In [1]:
import os
os.chdir('../quafing/')
print(f"Working directory: {os.getcwd()}")
import quafing as q


import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter


plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({"font.size" : 15, 
                     "figure.dpi" : 100, 
                     "legend.fontsize" : 13, 
                     "grid.alpha" : 0.3, 
                     "axes.grid": True, 
                     "axes.axisbelow" : True, 
                     "figure.figsize":(6, 5)})

Working directory: /Users/charlesdupont/Desktop/Thesis/code/quafing


In [2]:
def load_data(path):
    """
    Loads .dta file using provided path.
    """
    return pd.read_stata(path, convert_categoricals=False)

In [3]:
DATA_DIR = "../../BCCASII/"
SUB_DATA_DIR = "Household/"

In [4]:
all_hhid = set()
for i, filename in enumerate(os.listdir(DATA_DIR + SUB_DATA_DIR)):
    if ".dta" not in filename:
        continue
    data = load_data(DATA_DIR + SUB_DATA_DIR + filename)
    all_hhid = all_hhid.union(set(data["hhid"].unique()))

In [5]:
def get_dataframes():

    timea = time.time()

    all_dfs = {hhid:[] for hhid in all_hhid}

    for file_index, filename in enumerate(sorted(os.listdir(DATA_DIR + SUB_DATA_DIR))):

        if ".dta" in filename:

            data = load_data(DATA_DIR + SUB_DATA_DIR + filename)

            dataframes = {}
            max_cols = -1
            max_col_df = None

            for hhid in all_hhid:
                df = data.loc[data.hhid==hhid].drop(columns=["hhid"])  
                num_rows = df.shape[0]
                # create empty row if df is empty
                if num_rows == 0:
                    df = pd.DataFrame({col:[np.nan] for col in df.columns})
                series = [df.iloc[i].rename({col:f"{col}_{i}" for col in df.columns}) for i in range(len(df))]
                df = pd.DataFrame(pd.concat(series)).T

                dataframes[hhid] = df

                # keep track of df with most entries
                num_cols = df.shape[1]
                if num_cols > max_cols:
                    max_cols = num_cols
                    max_col_df = df

            for hhid, df in dataframes.items():
                supplementary = pd.Series({col:np.nan for col in max_col_df if col not in df}, dtype="object")
                all_dfs[hhid].append(pd.DataFrame(pd.concat([df.squeeze(), supplementary])).T)
   
    # combine all data
    for hhid, df in all_dfs.items():
        all_dfs[hhid] = pd.concat(df, axis=1)
    combined = pd.concat(all_dfs.values())
    combined["hhid"] = list(all_dfs.keys())
    
    print(f"Elapsed time: {time.time()-timea}")
    
    return combined

In [20]:
try:
    combined = pd.read_csv("../notebooks/data/combined_raw.csv")
except:
    combined = get_dataframes()

In [21]:
combined

,vcode_0,ucode_0,tcode_0,dcode_0,phone_0,ezcode_0,rid_male_0,rid_female_0,hhhid_0,religion_0,...,v03_10,v04_10,v05_10,mid_11.3,v01_11,v02_11,v03_11,v04_11,v05_11,hhid
0,NaN,1.0,1.0,1.0,,7.0,1.0,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,NaN,1.0,1.0,1.0,,7.0,1.0,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2,NaN,1.0,1.0,1.0,,7.0,1.0,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
3,NaN,1.0,1.0,1.0,,7.0,1.0,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
4,NaN,1.0,1.0,1.0,,7.0,NaN,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,NaN,34.0,33.0,28.0,,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,673.0
824,NaN,8.0,8.0,6.0,,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0
825,NaN,38.0,37.0,30.0,,6.0,11.0,12.0,11.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,744.2
826,NaN,38.0,37.0,30.0,,6.0,NaN,16.0,16.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,744.3


### Remove columns that have only null values, or identical values

In [60]:
L = len(combined)
to_remove = []
for col in combined.columns:
    col_values = combined[col]
    if sum(col_values.isnull()) == L or len(col_values.unique()) == 1:
        to_remove.append(col)

In [63]:
combined.drop(to_remove, axis=1, inplace=True)

In [74]:
combined

,ucode_0,tcode_0,dcode_0,phone_0,ezcode_0,rid_male_0,rid_female_0,hhhid_0,a14yy_0,a15yy_0,...,v05_8,mid_9.3,v03_9,v04_9,v05_9,mid_10.3,v05_10,mid_11.3,v05_11,hhid
0,1.0,1.0,1.0,,7.0,1.0,2.0,1.0,2012.0,2012.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1.0,1.0,1.0,,7.0,1.0,2.0,1.0,2012.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2,1.0,1.0,1.0,,7.0,1.0,2.0,1.0,2012.0,2012.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
3,1.0,1.0,1.0,,7.0,1.0,2.0,1.0,2012.0,2012.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
4,1.0,1.0,1.0,,7.0,NaN,2.0,1.0,2012.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,34.0,33.0,28.0,,1.0,NaN,NaN,NaN,2012.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,673.0
824,8.0,8.0,6.0,,3.0,NaN,NaN,NaN,2012.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0
825,38.0,37.0,30.0,,6.0,11.0,12.0,11.0,2012.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,744.2
826,38.0,37.0,30.0,,6.0,NaN,16.0,16.0,2012.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,744.3


In [195]:
def marginal_probs(df, column):
    """
    Computes marginal probabilities for all unique values appearing in a given column.
    Args:
        df:     dataframe of intereest
        column: column for which we want to compute the marginal probabilities
    Returns:
        marginal probabilities as dictionary index by column value
    """
    L = len(df)
    probs = {}
    unique_vals = df[column].unique()
    for value in unique_vals:
        probs[value] = sum(df[column]==value)/L
    return probs

In [213]:
mp = marginal_probs(combined, "ucode_0")
mp

{1.0: 0.024154589371980676,
 2.0: 0.024154589371980676,
 3.0: 0.024154589371980676,
 4.0: 0.026570048309178744,
 5.0: 0.026570048309178744,
 6.0: 0.026570048309178744,
 7.0: 0.024154589371980676,
 8.0: 0.025362318840579712,
 9.0: 0.026570048309178744,
 10.0: 0.030193236714975844,
 11.0: 0.026570048309178744,
 12.0: 0.024154589371980676,
 13.0: 0.024154589371980676,
 14.0: 0.024154589371980676,
 15.0: 0.024154589371980676,
 16.0: 0.025362318840579712,
 17.0: 0.026570048309178744,
 18.0: 0.024154589371980676,
 19.0: 0.027777777777777776,
 20.0: 0.024154589371980676,
 21.0: 0.025362318840579712,
 22.0: 0.024154589371980676,
 23.0: 0.024154589371980676,
 24.0: 0.025362318840579712,
 25.0: 0.024154589371980676,
 26.0: 0.024154589371980676,
 27.0: 0.024154589371980676,
 28.0: 0.024154589371980676,
 29.0: 0.024154589371980676,
 30.0: 0.024154589371980676,
 31.0: 0.024154589371980676,
 32.0: 0.024154589371980676,
 33.0: 0.024154589371980676,
 34.0: 0.024154589371980676,
 35.0: 0.02415458937198

In [196]:
combined_nan_replace = combined.replace(np.nan, "nan")

In [197]:
list(combined_nan_replace.iloc[0]) == list(combined_nan_replace.iloc[0])

True

In [198]:
marginals = {}
for col in combined_nan_replace:
    marginals[col] = marginal_probs(combined_nan_replace, col)

In [191]:
CC = Counter([tuple(combined_nan_replace.iloc[i]) for i in range(len(combined_nan_replace))])
len(CC)

828